In [1]:
import pandas as pd

# Load the Dataset

In [3]:
# Load data from one file for inspection
file_path = '../data/yfinance_data/AAPL_historical_data.csv'
data = pd.read_csv(file_path)

In [4]:
# Display the first few rows of the dataset
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,0.0,0.0
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,0.0,0.0
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,0.0,0.0
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,0.0,0.0
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,0.0,0.0


# Inspecting the Data
Check the structure of the data:

Columns: Ensure all the necessary columns (Date, Open, High, Low, Close, Adj Close, Volume, Dividends, Stock Splits) are present.


In [5]:
# Check data types and missing values
data.info()

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10998 entries, 0 to 10997
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          10998 non-null  object 
 1   Open          10998 non-null  float64
 2   High          10998 non-null  float64
 3   Low           10998 non-null  float64
 4   Close         10998 non-null  float64
 5   Adj Close     10998 non-null  float64
 6   Volume        10998 non-null  int64  
 7   Dividends     10998 non-null  float64
 8   Stock Splits  10998 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 773.4+ KB


### Data Types: 
Confirm that the data types are appropriate (e.g., Date should be a datetime object, numerical columns like Open, High, Close should be floats).
### Missing Values: 
Identify and handle any missing values.
### Date Format: 
Ensure the Date column is in the correct format and set it as the index.

In [6]:

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Check for any missing values
missing_values = data.isnull().sum()

# Display the missing values, if any
missing_values

Open            0
High            0
Low             0
Close           0
Adj Close       0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

# Handling Missing Values
If there are missing values, we need to decide on the best approach:

### Drop rows: 
If there are very few missing values, we may drop those rows.
## Fill values: 
 Alternatively, we could fill missing values with appropriate methods like forward fill, backward fill, or interpolation.

In [8]:
# Dropping rows with missing values
data.dropna(inplace=True)

# Or, filling missing values (example: forward fill)
data.ffill( inplace=True)


# Loading Data from All Files
Once the data from one file is properly loaded and cleaned, we can apply the same process to all files and concatenate them into a single DataFrame.

In [13]:
import os

# List of file paths
file_paths = [
    '../data/yfinance_data/AAPL_historical_data.csv',
    '../data/yfinance_data/AMZN_historical_data.csv',
    '../data/yfinance_data/GOOG_historical_data.csv',
    '../data/yfinance_data/META_historical_data.csv',
    '../data/yfinance_data//MSFT_historical_data.csv',
    '../data/yfinance_data/NVDA_historical_data.csv',
    '../data/yfinance_data/TSLA_historical_data.csv'
]

# Initialize an empty list to store dataframes
df_list = []

# Load and clean data from each file
for file in file_paths:
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df.ffill( inplace=True)
    df_list.append(df)

# Concatenate all dataframes into one
all_data = pd.concat(df_list, keys=[os.path.basename(file).split('_')[0] for file in file_paths], names=['Stock', 'Date'])


# Display the combined dataframe
all_data.head()


Open      High       Low     Close  Adj Close  \
Stock Date                                                            
AAPL  1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943   
      1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781   
      1980-12-16  0.113281  0.113281  0.112723  0.112723   0.086898   
      1980-12-17  0.115513  0.116071  0.115513  0.115513   0.089049   
      1980-12-18  0.118862  0.119420  0.118862  0.118862   0.091630   

                     Volume  Dividends  Stock Splits  
Stock Date                                            
AAPL  1980-12-12  469033600        0.0           0.0  
      1980-12-15  175884800        0.0           0.0  
      1980-12-16  105728000        0.0           0.0  
      1980-12-17   86441600        0.0           0.0  
      1980-12-18   73449600        0.0           0.0

# Calculating Basic Technical Indicators

Now that the data is loaded and cleaned, we'll use the TA-Lib library to calculate some basic technical indicators: Moving Averages, RSI (Relative Strength Index), and MACD (Moving Average Convergence Divergence).

## Calculating Moving Averages
Let's calculate the Simple Moving Average (SMA) and Exponential Moving Average (EMA) using pandas-ta.

###  Simple Moving Average (SMA)


In [33]:
# Calculate 20-day and 50-day SMA
all_data['SMA_20'] = all_data['Close'].rolling(window=20).mean()
all_data['SMA_50'] = all_data['Close'].rolling(window=50).mean()

#### Exponential Moving Average (EMA)

In [34]:
# Calculate 20-day and 50-day EMA
all_data['EMA_20'] = all_data['Close'].ewm(span=20, adjust=False).mean()
all_data['EMA_50'] = all_data['Close'].ewm(span=50, adjust=False).mean()

### Relative Strength Index (RSI)

In [35]:
# Calculate RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

all_data['RSI'] = calculate_rsi(all_data)

### Moving Average Convergence Divergence (MACD)

In [ ]:
# Calculate MACD
all['EMA_12'] = all['Close'].ewm(span=12, adjust=False).mean()
all['EMA_26'] = all['Close'].ewm(span=26, adjust=False).mean()
all['MACD'] = all['EMA_12'] - all['EMA_26']
all['MACD_Signal'] = all['MACD'].ewm(span=9, adjust=False).mean()